In [1]:
import pandas as pd
import pitcher
import sklearn

In [2]:
#currently these functions will only grab pitches CU, FF, SL, and CH so only use verlander and scherzer datasets
player = 'verlander'
data = pd.read_csv('Data/raw_data/'+player+'.csv')
data = pitcher.clean_data(data)
ABs = pitcher.get_abs(data)
reps = pitcher.get_reps(ABs)
reps = pitcher.drop_nas(reps)

/home/zac/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (87,88) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [3]:
data = []
for rep in reps:
    row = []
    prev_pitches,prev_types,pre_pitch,ptype = rep
    prevs = []
    for prev in prev_pitches:
        prevs += prev
    row += prevs + pre_pitch + [ptype]
    data.append(row)

running the cell above to get 'data' gives a list of rows each of length 91, which represents the 3 length 25 vectors plus the one length 15 vector plus the pitch type, i.e. the inputs to the network.

Each row of data now consists of:
- three previous pitch vectors (length 25) of at-bat, if the pitch-to-be-predicted is the first, second, or third pitch the at-bat, the previous empty spots will be filled with zeroes.
- one game state vector (length 15) representing the game state of the pitch-to-be-predicted
- the pitch-to-be-predicted 'NF' or 'FF', not-fastball or fourseam-fastball

In [4]:
#get train and test data
length = len(data[0]) - 1
Y = []
X = []
for row in data:
    X.append(row[75:length])
    if row[length] == 'NF':
        Y.append(0)
    else:
        Y.append(1)
X_train = X[:int(len(X)*0.8)]
X_test = X[int(len(X)*0.8):]
Y_train = Y[:int(len(Y)*0.8)]
Y_test = Y[int(len(Y)*0.8):]

In [5]:
#Decision Tree - works best with just game state 
from sklearn import tree
clf = tree.DecisionTreeClassifier()
clf.fit(X_train, Y_train)
clf.predict_proba(X_test)
print("*********DECISION TREE RESULTS*************")
print("Accuracy above naive guess:",clf.score(X_test, Y_test),clf.score(X_test, Y_test) - sum(Y_test)/len(Y_test))

*********DECISION TREE RESULTS*************
Accuracy above naive guess: 0.605369807497467 0.020516717325227973


In [6]:
#rest of models will use full input
length = len(data[0]) - 1
Y = []
X = []
for row in data:
    X.append(row[0:length])
    if row[length] == 'NF':
        Y.append(0)
    else:
        Y.append(1)
X_train = X[:int(len(X)*0.8)]
X_test = X[int(len(X)*0.8):]
Y_train = Y[:int(len(Y)*0.8)]
Y_test = Y[int(len(Y)*0.8):]

In [7]:
#Logistic Regression
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0, solver='lbfgs',multi_class='ovr').fit(X_train, Y_train)
rslt = clf.predict_proba(X_test)
print("*********LOGISTIC REGRESSION RESULTS*************")
print("Accuracy above naive guess:",clf.score(X_test, Y_test),clf.score(X_test, Y_test) - sum(Y_test)/len(Y_test))

*********LOGISTIC REGRESSION RESULTS*************
Accuracy above naive guess: 0.6388044579533941 0.05395136778115506


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


In [8]:
#SVM
from sklearn import svm
clf = svm.SVC(gamma='scale').fit(X_train, Y_train)
rslt = clf.predict(X_test)
right = 0
for pred,tar in zip(rslt,Y_test):
    if pred == tar:
        right += 1
print("*********SVM RESULTS*************")
print("Accuracy above naive guess:",right/len(rslt),right/len(rslt) - sum(Y_test)/len(Y_test))

*********SVM RESULTS*************
Accuracy above naive guess: 0.6152482269503546 0.03039513677811556
